In [ ]:
import graphene
from django.db import models
from graphene import Schema

from graphene_django import DjangoObjectType
from graphene_django.rest_framework.mutation import SerializerMutation
from rest_framework import serializers


In [ ]:
class MyFakeChildModel(models.Model):
    name = models.CharField(max_length=50)
    created = models.DateTimeField(auto_now_add=True)


In [ ]:
class MyFakeParentModel(models.Model):
    name = models.CharField(max_length=50)
    created = models.DateTimeField(auto_now_add=True)
    child1 = models.OneToOneField(
        MyFakeChildModel, related_name="parent1", on_delete=models.CASCADE
    )
    child2 = models.OneToOneField(
        MyFakeChildModel, related_name="parent2", on_delete=models.CASCADE
    )


In [ ]:
class ParentType(DjangoObjectType):
    class Meta:
        model = MyFakeParentModel
        interfaces = (graphene.relay.Node,)
        fields = "__all__"


In [ ]:
class ChildType(DjangoObjectType):
    class Meta:
        model = MyFakeChildModel
        interfaces = (graphene.relay.Node,)
        fields = "__all__"


In [ ]:
class MyModelChildSerializer(serializers.ModelSerializer):
    class Meta:
        model = MyFakeChildModel
        fields = "__all__"


In [ ]:
class MyModelParentSerializer(serializers.ModelSerializer):
    child1 = MyModelChildSerializer()
    child2 = MyModelChildSerializer()

    class Meta:
        model = MyFakeParentModel
        fields = "__all__"


In [ ]:
class MyParentModelMutation(SerializerMutation):
    class Meta:
        serializer_class = MyModelParentSerializer


In [ ]:
class Mutation(graphene.ObjectType):
    createParentWithChild = MyParentModelMutation.Field()


In [ ]:
def test_create_schema():
    schema = Schema(mutation=Mutation, types=[ParentType, ChildType])
    assert schema

